In [36]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.utils import check_random_state
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.linear_model import Lasso

import pandas as pd
import numpy as np

In [23]:
# try again with log sales rank as the target variable
data = pd.read_csv('./data/final_regression_dataset.csv')
# remove asins
data = data.drop('ASIN',axis=1)
# one hot encoding for any categorical vars
data = pd.get_dummies(data)
y = data.loc[:, data.columns == 'salesRank']
# y = y["salesRank"].tolist() 
# # apply log to the sales rank column
# y = y.apply(np.log)
# keep rest of the variables as it is
x = data.loc[:, data.columns != 'salesRank']
# split 75% for training and 25% for testing
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=1)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
y_train = y_train["salesRank"].tolist() 
y_test = y_test["salesRank"].tolist()

(15179, 404) (15179, 1) (5060, 404) (5060, 1)


In [25]:
base_estimator = DecisionTreeRegressor().fit(X_train, y_train)

# make predictions using DecisionTreeRegressor
y_pred = base_estimator.predict(X_test)
rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
print("RMSE in terms of sales rank = ",rmse, "R2 = ", base_estimator.score(X_train,y_train))

RMSE in terms of sales rank =  78442.4170835 R2 =  1.0


In [28]:
# Next try ensembling
rng = check_random_state(0)

# without bootstrap, all trees are perfect on the training set
model = BaggingRegressor(base_estimator=DecisionTreeRegressor(),
                            max_samples=1.0,
                            bootstrap=False,
                            random_state=rng).fit(X_train, y_train)
# make predictions using Bagging Regressor
y_pred = model.predict(X_test)
rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
print("RMSE in terms of sales rank = ",rmse, "R2 = ", model.score(X_train,y_train))

RMSE in terms of log(sales rank) =  73141.7753639 R2 =  1.0


In [31]:
# ensemble with bootstraping and more estimators
model = BaggingRegressor(base_estimator=DecisionTreeRegressor(),
                            max_samples=1.0,
                            bootstrap=True,
                            random_state=rng,
                            n_estimators=250
                        ).fit(X_train, y_train)
# make predictions using Bagging Regressor
y_pred = model.predict(X_test)
rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
print("RMSE in terms of log(sales rank) = ",rmse, "R2 = ", model.score(X_train,y_train))

RMSE in terms of log(sales rank) =  56886.0274835 R2 =  0.860026387242


In [34]:
# try random forests
model = RandomForestRegressor(n_estimators=250)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
print("RMSE in terms of log(sales rank) = ",rmse, "R2 = ", model.score(X_train,y_train))

RMSE in terms of log(sales rank) =  57016.7174201 R2 =  0.859433766997


In [65]:
# try reducing the features to those selected by lasso
# for alpha = 0.00008 we get RMSE = 1.45 while removing 266 features
# Lets look at the features not removed by Lasso

data = pd.read_csv('./data/final_regression_dataset.csv')
# remove asins
data = data.drop('ASIN',axis=1)
# one hot encoding for any categorical vars
data = pd.get_dummies(data)
y = data.loc[:, data.columns == 'salesRank']
# apply log to the sales rank column
y = y.apply(np.log)
# keep rest of the variables as it is
x = data.loc[:, data.columns != 'salesRank']
names = x.columns.tolist()
lasso = Lasso(alpha=0.00008,normalize=True, max_iter=10000)
lasso.fit(x, y)
y_pred = lasso.predict(x)

rmse = np.sqrt(metrics.mean_squared_error(y, y_pred))
print("RMSE in terms of log(sales rank) = ",rmse)
print("in terms of normal sales rank this is = ", np.e**(rmse))
goodFeaturesDict = {}
for n,c in zip(names, lasso.coef_):
    if c != 0.0:
        goodFeaturesDict[n] = c

RMSE in terms of log(sales rank) =  1.45882400431
in terms of normal sales rank this is =  4.30089871484


In [67]:
# use features selected by authority
x = x[[*goodFeaturesDict.keys()]]
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=1)
y_train = y_train["salesRank"].tolist() 
y_test = y_test["salesRank"].tolist()

# try random forests again
model = RandomForestRegressor(n_estimators=100)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
print("RMSE in terms of log(sales rank) = ",rmse, 
      "in terms of normal sales rank this is = ", np.e**(rmse), 
      "R2 = ", model.score(X_train,y_train))

RMSE in terms of log(sales rank) =  1.48258962266 in terms of normal sales rank this is =  4.40433649517 R2 =  0.867880618586


In [69]:
# read the entire dataset
data = pd.read_csv("./data/final_regression_dataset.csv")
print(data.shape)
# split data into dataframes for each category
data.sort_values(by = ['category'], inplace=True)
# set the index to be this and don't drop
data.set_index(keys = ['category'], drop=False,inplace=True)
# get a list of all product categories
categories = data['category'].unique().tolist()
print(categories)
df = data.groupby('category').size()

(20239, 403)
['Beauty', 'Grocery & Gourmet Food', 'Health & Personal Care', 'Sports &amp; Outdoors']


/Users/megh/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:11: FutureWarning: 'category' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version
